# Test Acc 성능을 높이기 위한 방안
- Derived Variable 생성해서 다시 학습
    - 결과: 시계열 데이터 피쳐 추출 (aeon : 시계열 데이터 Classification - Feature Based 라이브러리 사용)
        - Catch 22 (이상치 비율, 자기 상관지수(autoCorrelation), ...) 성능: 0.549
        - SummaryClassifier (평균, 편차, ...) : 0.592
- Model 변경
    - 결과: 유의미한 결과 없음
    - 성능은 0.69를 넘지 못함
- 최적화 (Grid Search)
    - 결과: 유의미한 결과 없음
    - 성능은 0.69를 넘지 못함

# 모델 변경
추가) 클래스 불균형 반영 : 07/30
- LightGBM
- CatBoost
- XGBoost

In [3]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from lightgbm import DaskLGBMClassifier, LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

# 윈도우 기준 예시 (맑은 고딕)
rc('font', family='Malgun Gothic')
def init():
    walk_target = pd.read_csv('LifeLog\Data\\1.Training\라벨링데이터\\1.걸음걸이\\training_label.csv').rename(columns={'SAMPLE_EMAIL':"EMAIL"})
    walk_lifelog = pd.read_csv('LifeLog\Data\\1.Training\원천데이터\\1.걸음걸이\\train_activity.csv')
    walk_df = walk_lifelog.merge(walk_target,on='EMAIL')
    walk_df.rename(columns={
        'activity_average_met': '하루간 평균 MET',
        'activity_cal_active': '하루간 활동 칼로리',
        'activity_cal_total': '하루간 총 사용 칼로리',
        'activity_class_5min': '하루간 5분당 활동 로그',
        'activity_daily_movement': '매일 움직인 거리',
        'activity_day_end': '활동 종료 시간',
        'activity_day_start': '활동 시작 시간',
        'activity_high': '고강도 활동 시간',
        'activity_inactive': '비활동 시간',
        'activity_inactivity_alerts': '비활동 알람 횟수',
        'active_low': '저강도 활동 시간',
        'activity_medium': '중강도 활동 시간',
        'activity_met_1min': '하루간 1분 당 MET 로그',
        'activity_met_min_high': '하루간 고강도 활동 MET',
        'activity_met_min_inactive': '하루간 비활동 MET',
        'activity_met_min_low': '하루간 저강도 활동 MET',
        'activity_met_min_medium': '하루간 중강도 활동 MET',
        'activity_non_wear': '미착용 시간',
        'activity_rest': '휴식 시간',
        'activity_score': '활동 점수',
        'activity_low':'저강도 활동 시간',
        'activity_score_meet_daily_targets': '활동 목표달성 점수',
        'activity_score_move_every_hour': '매 시간 당 활동유지 점수',
        'activity_score_recovery_time': '회복시간 점수',
        'activity_score_stay_active': '활동 유지 점수',
        'activity_score_training_frequency': '운동 빈도 점수',
        'activity_score_training_volume': '운동량 점수',
        'activity_steps': '매일 걸음 수',
        'activity_total': '활동 총 시간(분)',
        'CONVERT(activity_class_5min USING utf8)': '하루간 5분당 활동 로그 (UTF-8)',
        'CONVERT(activity_met_1min USING utf8)': '하루간 1분당 MET 로그 (UTF-8)',
        'EMAIL': '이메일',
        'DIAG_NM': '진단명'
    }, inplace=True)
    return walk_df

selected_columns = ['하루간 평균 MET','하루간 활동 칼로리','하루간 총 사용 칼로리','비활동 시간','비활동 알람 횟수','저강도 활동 시간','하루간 비활동 MET',
                    '하루간 저강도 활동 MET','활동 점수','활동 목표달성 점수','회복시간 점수','활동 유지 점수','운동량 점수','매일 걸음 수','활동 총 시간(분)','진단명']

def feature_selection(df, columns:list):
    return df[columns]

def separate(df):
    X = df.drop(columns='진단명')
    y = df[['진단명']]
    le = LabelEncoder()
    y = le.fit_transform(y)
    return train_test_split(X,y,test_size=0.2,stratify=y)

def train_and_evaluate(model, X_train, y_train, X_test, y_test, scale=False):
    if scale:
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        pipeline.fit(X_train, y_train)
        train_score = pipeline.score(X_train, y_train)
        test_score = pipeline.score(X_test, y_test)
        feature_importances = None
    else:
        model.fit(X_train, y_train)
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)
        feature_importances = getattr(model, 'feature_importances_', None)
    print(f"Model: {model}")
    print(f"Train accuracy: {train_score:.4f}")
    print(f"Test accuracy: {test_score:.4f}")
    if feature_importances is not None:
        print("Feature importances:", feature_importances)
    print(f'feature_names: {X_train.columns.tolist()}')
    print()

    return {'feature_importances': feature_importances, 'train_score':train_score, 'test_score':test_score, 'feature_names': X_train.columns.tolist()}

def plot_feature_importances(feature_importances, feature_names, model_name):
    if feature_importances is None:
        print(f"{model_name}에는 feature_importances가 없습니다.")
        return
    
    indices = np.argsort(feature_importances)[::-1]  # 중요도 내림차순 정렬
    
    plt.figure(figsize=(8,4))
    plt.title(f"{model_name} Feature Importances")
    plt.bar(range(len(feature_importances)), feature_importances[indices], align='center')
    plt.xticks(range(len(feature_importances)), [feature_names[i] for i in indices], rotation=90)
    plt.ylabel("Importance")
    plt.tight_layout()
    plt.show()

In [4]:
df = init()
selected_df = feature_selection(df, selected_columns)
X_train, X_test, y_train, y_test = separate(selected_df)
RF_result = train_and_evaluate(RandomForestClassifier(class_weight='balanced',n_estimators=200), X_train, y_train, X_test,y_test)
GBC_result = train_and_evaluate(GradientBoostingClassifier(), X_train, y_train, X_test,y_test)
Ada_result = train_and_evaluate(AdaBoostClassifier(), X_train, y_train, X_test,y_test)
KNN_result = train_and_evaluate(KNeighborsClassifier(), X_train, y_train, X_test, y_test, scale=True)
SVC_result = train_and_evaluate(SVC(class_weight='balanced'), X_train, y_train, X_test, y_test, scale=True)
DT_result = train_and_evaluate(DecisionTreeClassifier(class_weight='balanced'), X_train, y_train, X_test, y_test)
XGBRF_result = train_and_evaluate(XGBRFClassifier(), X_train, y_train, X_test, y_test)
XGB_result = train_and_evaluate(XGBClassifier(), X_train, y_train, X_test, y_test)
LGBM_result = train_and_evaluate(LGBMClassifier(class_weight='balanced'), X_train, y_train, X_test, y_test)
CatBoost_result = train_and_evaluate(CatBoostClassifier(), X_train, y_train, X_test, y_test)

Model: RandomForestClassifier(class_weight='balanced', n_estimators=200)
Train accuracy: 1.0000
Test accuracy: 0.6574
Feature importances: [0.04700879 0.08232734 0.09515454 0.08148083 0.01768976 0.08926411
 0.05808321 0.09415463 0.06259991 0.05827243 0.02282461 0.06853633
 0.05978615 0.08303718 0.07978017]
feature_names: ['하루간 평균 MET', '하루간 활동 칼로리', '하루간 총 사용 칼로리', '비활동 시간', '비활동 알람 횟수', '저강도 활동 시간', '하루간 비활동 MET', '하루간 저강도 활동 MET', '활동 점수', '활동 목표달성 점수', '회복시간 점수', '활동 유지 점수', '운동량 점수', '매일 걸음 수', '활동 총 시간(분)']

Model: GradientBoostingClassifier()
Train accuracy: 0.7005
Test accuracy: 0.6455
Feature importances: [0.02725147 0.05270416 0.18789377 0.05801902 0.05200414 0.07077345
 0.06452291 0.06061656 0.063974   0.07311168 0.02135609 0.06750089
 0.08142507 0.07525114 0.04359564]
feature_names: ['하루간 평균 MET', '하루간 활동 칼로리', '하루간 총 사용 칼로리', '비활동 시간', '비활동 알람 횟수', '저강도 활동 시간', '하루간 비활동 MET', '하루간 저강도 활동 MET', '활동 점수', '활동 목표달성 점수', '회복시간 점수', '활동 유지 점수', '운동량 점수', '매일 걸음 수', '활동 총 시간(분)']


# 모델 변경에 대한 결론
- 모델의 한계성 (시계열성의 데이터를 이해할 수 없다.)

이유: 

    Tree 기반의 모델인 (Decision Tree, RandomForest, LightGBM, GradientBoosting, XGBoost)들은 시계열 데이터를 각각 개별의 독립적인 데이터로 인식하게 되어, 시계열성을 배제하게 된다. 따라서, 이러한 시계열성의 데이터를 반영할 수 있는 딥러닝 모델인 (RNN 계열의 모델: RNN, LSTM, GRU 모델) 또는 Attention 기반의 모델인 BERT, Transformer를 사용해 데이터를 분류할 필요성이 보인다.

결론:

    - 데이터들은 이미 하루 단위로 요약 피쳐로 존재하기 때문에, 더 이상 시계열성의 Feature를 파생변수를 만들 필요성이 없다고 판단했다. 
    다른 데이터셋을 추가하여, 치매를 분류할 수 있는 좋은 변수를 더하는 것이 좋겠다고 판단을 했다.

    - 머신러닝으로는 분류 성능이 저조하다.

    - Grid Search 진행 (clear)

    - ~Bayesian Model로 평균을 사용해 분류를 진행하는 것을 시도하고, 이마저도 성능이 낮다면, 데이터셋을 추가하는 것으로 한다.~


# 최적화 진행해 성능 향상 도모

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None],
}

rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=3, scoring='accuracy')
rf_grid.fit(X_train, y_train)
rf_best = rf_grid.best_estimator_

# 2. XGBoost
xgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6],
    'learning_rate': [0.1, 0.01]
}
xgb_grid = GridSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
                        xgb_params, cv=3, scoring='accuracy')
xgb_grid.fit(X_train, y_train)
xgb_best = xgb_grid.best_estimator_

# 3. LightGBM
lgbm_params = {
    'n_estimators': [100, 200],
    'max_depth': [-1, 10],
    'learning_rate': [0.1, 0.01]
}
lgbm_grid = GridSearchCV(LGBMClassifier(random_state=42), lgbm_params, cv=3, scoring='accuracy')
lgbm_grid.fit(X_train, y_train)
lgbm_best = lgbm_grid.best_estimator_

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000330 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2121
[LightGBM] [Info] Number of data points in the train set: 5176, number of used features: 15
[LightGBM] [Info] Start training from score -0.517805
[LightGBM] [Info] Start training from score -2.834760
[LightGBM] [Info] Start training from score -1.062935
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000455 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2119
[LightGBM] [Info] Number of data points in the train set: 5176, number of used features: 15
[LightGBM] [Info] Start training from score -0.518129
[LightGBM] [Info] Start training from score -2.8

In [6]:
rf_pred = rf_best.predict(X_test)
print(np.equal(rf_pred,y_test).sum()/len(rf_pred))

lgbm_pred = lgbm_best.predict(X_test)
print(np.equal(lgbm_pred,y_test).sum()/len(lgbm_pred))

xgb_pred = xgb_best.predict(X_test)
print(np.equal(xgb_pred,y_test).sum()/len(xgb_pred))

0.6584234930448223
0.6568778979907264
0.657908294693457


In [23]:
rf_best.predict_proba(X_test)[0]

array([0.76, 0.06, 0.18])

# 시계열 데이터 
- SummaryClassifier => (mean, std, min, max, 0.25, 0.5, 0.75)를 추출해서 분류 모델 예측
- Catch22 => 22개의 Canonical한 특성 (주기성, 계절성, 변동성, ...)

## Summary Classifier

In [7]:
from aeon.classification.feature_based import SummaryClassifier
clf = SummaryClassifier(n_jobs=-1, estimator=RandomForestClassifier(n_estimators=5))
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
np.equal(y_pred,y_test).sum()/len(lgbm_pred)

np.float64(0.5435342606903658)

## Catch22

In [15]:
from aeon.transformations.collection.feature_based import Catch22
from aeon.classification.feature_based import Catch22Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import numpy as np

# Transformer 적용
transformer = Catch22(n_jobs=-1)
X_feat_train = transformer.fit_transform(X_train, y_train)
X_feat_test = transformer.transform(X_test)    # y_test 제거

# 결측치 처리 (필요 시)
imputer = SimpleImputer(strategy='mean')
X_feat_train = imputer.fit_transform(X_feat_train)
X_feat_test = imputer.transform(X_feat_test)

# classifier 학습
clf = RandomForestClassifier()
clf.fit(X_feat_train, y_train)

# 예측 및 평가
y_pred = clf.predict(X_feat_test)
acc = np.mean(y_pred == y_test)
print("Accuracy:", acc)

Accuracy: 0.592478104070067


In [16]:
X_feat_train

array([[3.52550000e+03, 1.76275000e+03, 5.83853187e-01, ...,
        1.71049323e+00, 1.57079633e+00, 1.10136913e+04],
       [5.84100000e+02, 2.92050000e+02, 5.79379497e-01, ...,
        2.00755631e+00, 1.57079633e+00, 1.85279817e+03],
       [3.54100000e+02, 1.77050000e+02, 5.83526827e-01, ...,
        1.77213939e+00, 1.57079633e+00, 1.16677825e+03],
       ...,
       [1.50840000e+03, 7.54200000e+02, 5.83635891e-01, ...,
        1.97018144e+00, 1.57079633e+00, 4.70064891e+03],
       [2.26490000e+03, 1.13245000e+03, 5.82396910e-01, ...,
        1.77213939e+00, 1.57079633e+00, 7.09658201e+03],
       [8.17565625e+02, 4.09485937e+02, 5.80660779e-01, ...,
        1.77213939e+00, 1.57079633e+00, 2.57729558e+03]], shape=(7764, 20))

# 베이지안 모델 (시계열성이 포함된)

Bayesian Model로 평균을 사용해 분류를 진행하는 것을 시도하고, 이마저도 성능이 낮다면, 데이터셋을 추가하는 것으로 한다.

In [18]:
selected_df.groupby('진단명').mean()

,하루간 평균 MET,하루간 활동 칼로리,하루간 총 사용 칼로리,비활동 시간,비활동 알람 횟수,저강도 활동 시간,하루간 비활동 MET,하루간 저강도 활동 MET,활동 점수,활동 목표달성 점수,회복시간 점수,활동 유지 점수,운동량 점수,매일 걸음 수,활동 총 시간(분)
진단명,,,,,,,,,,,,,,,
CN,1.471274,466.329182,2490.928905,523.649196,0.509600,282.430029,7.784120,192.782737,82.476734,76.198409,85.363259,77.185089,87.640720,10666.965577,338.620308
Dem,1.383483,400.504378,2425.490368,479.168126,0.381786,213.781086,6.464098,149.569177,74.486865,46.844133,90.357268,81.656743,76.739054,8788.700525,262.921191
MCI,1.484892,483.042052,2544.159559,497.246048,0.330152,310.628691,8.229943,209.276171,84.698777,79.360274,86.942440,80.343275,89.056367,11186.545183,366.751864


In [24]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 예시용: 전체 raw data가 있다고 가정 (selected_df)
X = selected_df.drop(columns=['진단명'])
y = selected_df['진단명']

# NaN 제거 또는 보간
X = X.fillna(X.mean())

# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# 베이지안 모델 학습
clf = GaussianNB()
clf.fit(X_train, y_train)

# 예측 및 평가
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
selected_df

              precision    recall  f1-score   support

          CN       0.63      0.47      0.54      1446
         Dem       0.19      0.38      0.26       143
         MCI       0.41      0.53      0.46       838

    accuracy                           0.48      2427
   macro avg       0.41      0.46      0.42      2427
weighted avg       0.53      0.48      0.49      2427



,하루간 평균 MET,하루간 활동 칼로리,하루간 총 사용 칼로리,비활동 시간,비활동 알람 횟수,저강도 활동 시간,하루간 비활동 MET,하루간 저강도 활동 MET,활동 점수,활동 목표달성 점수,회복시간 점수,활동 유지 점수,운동량 점수,매일 걸음 수,활동 총 시간(분),진단명
0,1.28125,196,2251,714,0,201,8,133,87,78,100,54,97,4212,207,MCI
1,1.25000,145,2159,719,0,131,9,82,83,60,100,58,96,3306,140,MCI
2,1.21875,118,2140,708,1,125,12,66,73,43,100,55,83,2550,131,MCI
3,1.28125,180,2240,705,0,203,10,122,70,43,100,58,59,4009,208,MCI
4,1.46875,374,2559,437,0,511,18,268,70,43,100,89,46,8081,517,MCI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,1.34375,249,2351,780,4,167,10,106,61,25,100,45,95,4972,190,CN
9701,1.53125,570,2682,518,1,332,7,257,78,43,100,76,93,11878,394,CN
9702,1.34375,295,2331,578,0,173,8,99,70,25,100,70,95,5752,207,CN
9703,1.31250,186,2280,778,1,138,11,86,66,25,100,47,93,3852,156,CN
